In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

## Load dataset

In [2]:
path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00597/garments_worker_productivity.csv'

In [3]:
df = pd.read_csv(path)

In [4]:
df.sample(n=10)

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
531,1/31/2015,Quarter5,finishing,Saturday,6,0.60,3.94,NaN,1200,0,0.0,0,0,10.0,0.971867
993,3/1/2015,Quarter1,sweing,Sunday,10,0.70,21.82,1393.0,5640,30,0.0,0,0,52.0,0.700237
664,2/8/2015,Quarter2,sweing,Sunday,12,0.80,15.26,1026.0,2880,63,0.0,0,0,34.0,0.800355
554,2/1/2015,Quarter1,sweing,Sunday,12,0.75,15.26,1276.0,1440,45,0.0,0,0,35.0,0.750451
900,2/23/2015,Quarter4,finishing,Monday,9,0.80,2.90,NaN,960,0,0.0,0,0,8.0,0.371563
825,2/18/2015,Quarter3,finishing,Wednesday,2,0.80,5.13,NaN,1440,0,0.0,0,0,12.0,0.919125
116,1/7/2015,Quarter1,sweing,Wednesday,6,0.80,11.41,986.0,5490,50,0.0,0,0,30.5,0.800344
961,2/26/2015,Quarter4,sweing,Thursday,1,0.35,26.66,1164.0,6600,23,0.0,0,2,55.0,0.378895
332,1/19/2015,Quarter3,sweing,Monday,7,0.80,48.84,1040.0,10260,30,0.0,0,0,57.0,0.576460
594,2/3/2015,Quarter1,sweing,Tuesday,8,0.70,24.26,1363.0,6660,0,0.0,0,0,55.5,0.586465


In [5]:
df.shape

(1197, 15)

Bộ dữ liệu có 1197 sample và 15 feature.

## Data wrangling

Kiểm tra kiểu dữ liệu của từng cột.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

Chuyển kiểu của 'date' về dạng datetime.

In [7]:
df['date'] = pd.to_datetime(df['date'])

In [8]:
df['date'].sample(n=5)

343   2015-01-20
990   2015-03-01
186   2015-01-11
866   2015-02-22
251   2015-01-14
Name: date, dtype: datetime64[ns]

Xem các giá trị khác nhau của các biến kiểu object.

In [9]:
df['quarter'].unique()

array(['Quarter1', 'Quarter2', 'Quarter3', 'Quarter4', 'Quarter5'],
      dtype=object)

In [10]:
df['department'].unique()

array(['sweing', 'finishing ', 'finishing'], dtype=object)

Có 2 department là 'sweing' (bị viết sai chính tả) và 'finishing' (có 1 từ bị dư dấu cách cuối). Sửa lại cho đúng.

In [11]:
df['department'] = df['department'].apply(lambda x: 'sewing' if x == 'sweing' else 'finishing')

In [12]:
df['department'].unique()

array(['sewing', 'finishing'], dtype=object)

In [13]:
df['day'].unique()

array(['Thursday', 'Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday'],
      dtype=object)

Trong df['day'] không có 'Friday', các công nhân được nghỉ ngày thứ sáu.

In [14]:
df['team'].unique()

array([ 8,  1, 11, 12,  6,  7,  2,  3,  9, 10,  5,  4], dtype=int64)

Biến 'team' mang tính phân loại (categorical), vì ý nghĩa của nó là số thứ tự của team chứ không mang ý nghĩa về mặt giá trị số. Do đó chuyển về 'object' và phân tích như biến categorical.

In [15]:
df['team'] = df['team'].apply(lambda x: 'Team'+str(int(x)))

Kiểm tra giá trị null.

In [16]:
df.isna().sum(axis=0)

date                       0
quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64

Feature 'wip' có tới 506 giá trị null, ta điền giá trị khuyết bằng phương pháp kNN với thư viện sklearn.

In [17]:
from sklearn.impute import KNNImputer

imputer = KNNImputer()
df[df.columns[5:]] = imputer.fit_transform(df.iloc[:,5:])

In [18]:
df['wip'].isna().sum()

0

In [19]:
df['wip']

0       1108.0
1        994.4
2        968.0
3        968.0
4       1170.0
         ...  
1192     994.4
1193     994.4
1194     994.4
1195     632.8
1196     926.0
Name: wip, Length: 1197, dtype: float64

## Wrapping in a single function

In [20]:
def data_loader(path='https://archive.ics.uci.edu/ml/machine-learning-databases/00597/garments_worker_productivity.csv',
                preprocessing=False):
    df = pd.read_csv(path)
    if preprocessing == True:
        df['date'] = pd.to_datetime(df['date'])
        df['department'] = df['department'].apply(lambda x: 'sewing' if x == 'sweing' else 'finishing')
        df['team'] = df['team'].apply(lambda x: 'Team'+str(int(x)))
        from sklearn.impute import KNNImputer
        imputer = KNNImputer()
        df[df.columns[5:]] = imputer.fit_transform(df.iloc[:,5:])
    return df

In [21]:
df1 = data_loader(preprocessing=True)

In [22]:
df1.dtypes

date                     datetime64[ns]
quarter                          object
department                       object
day                              object
team                             object
targeted_productivity           float64
smv                             float64
wip                             float64
over_time                       float64
incentive                       float64
idle_time                       float64
idle_men                        float64
no_of_style_change              float64
no_of_workers                   float64
actual_productivity             float64
dtype: object

In [23]:
df1.isna().sum(axis=0)

date                     0
quarter                  0
department               0
day                      0
team                     0
targeted_productivity    0
smv                      0
wip                      0
over_time                0
incentive                0
idle_time                0
idle_men                 0
no_of_style_change       0
no_of_workers            0
actual_productivity      0
dtype: int64